In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import skimage
import imgaug
import imgaug.augmenters as iaa

from tqdm import tqdm


# Root directory of the project
ROOT_DIR = os.path.abspath("./")

from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from moles_large import MolesConfig
from moles_large import ISIC17Dataset
from moles_large import ISIC17AugDataset

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


In [2]:
config = MolesConfig()

In [3]:
dataset_train = ISIC17AugDataset()
dataset_train.load_moles("data/large/train/")
dataset_train.prepare()
ds = dataset_train

dataset_val = ISIC17Dataset()
dataset_val.load_moles("data/large/val")
dataset_val.prepare()

In [ ]:
len(ds.image_ids)

In [ ]:
ds = dataset_train

# Load and display random samples
image_ids = np.random.choice(ds.image_ids, 4)
for image_id in image_ids:
    image = ds.load_image(image_id)
    mask, class_ids = ds.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, ds.class_names)

In [4]:
augmentation = iaa.SomeOf((4,5), [
    iaa.Fliplr(1.0),
    iaa.OneOf([
        iaa.GaussianBlur((1, 6.0)),
        iaa.AverageBlur(k=(1, 3)),
        iaa.MedianBlur(k=(3, 5)),
    ]),
    iaa.Flipud(1.0),
    iaa.Rot90((1,3)),
    iaa.CropAndPad(percent=(-0.25, 0.25)),
    iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.2),
    iaa.OneOf([iaa.Dropout((0.01, 0.1), per_channel=0.5),
               iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),]),
    iaa.PiecewiseAffine(scale=(0.01, 0.05)),
    iaa.LinearContrast((0.75, 1.5)),
])

In [5]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
box_ind is deprecated, use box_indices instead


In [6]:
COCO_MODEL_PATH = ROOT_DIR + "/mrcnn/mask_rcnn_coco.h5"
LATEST_MODEL_PATH = ROOT_DIR + "/logs/moles20200728T1523/mask_rcnn_moles_0160.h5"

LATEST = True

if LATEST:
    model.load_weights(LATEST_MODEL_PATH, by_name=True)
else:
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                "mrcnn_bbox", "mrcnn_mask"])


model.keras_model.metrics_tensors = []

Re-starting from epoch 160


In [7]:
# Train top layers
model.train(ds, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=30, 
            layers='heads',
            augmentation=augmentation
            )


Starting at epoch 3. LR=0.001

Checkpoint Path: /home/useydaninja/projects/melanoma/Skin-Cancer-Segmentation/logs/moles20200728T1523/mask_rcnn_moles_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_con

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "







Epoch 4/30
400/400 [==============================] - 1137s 3s/step - loss: 2.7083 - val_loss: 2.3763

Epoch 5/30
400/400 [==============================] - 802s 2s/step - loss: 2.5978 - val_loss: 2.0494
Epoch 6/30
400/400 [==============================] - 993s 2s/step - loss: 2.4902 - val_loss: 2.0508
Epoch 7/30
400/400 [==============================] - 993s 2s/step - loss: 2.4553 - val_loss: 1.3041
Epoch 8/30
400/400 [==============================] - 970s 2s/step - loss: 2.4153 - val_loss: 1.8773
Epoch 9/30
400/400 [==============================] - 990s 2s/step - loss: 2.3448 - val_loss: 2.1709
Epoch 10/30
400/400 [==============================] - 978s 2s/step - loss: 2.2776 - val_loss: 1.6891
Epoch 11/30
400/400 [==============================] - 983s 2s/step - loss: 2.2142 - val_loss: 2.1138
Epoch 12/30
400/400 [==============================] - 980s 2s/step - loss: 2.2527 - val_loss: 1.9611
Epoch 13/30
400/400 [==============================] - 982s 2s/step - loss: 2.177

In [7]:
# Train middle + top
model.train(ds, dataset_val,
            learning_rate=config.LEARNING_RATE/3,
            epochs=90,
            augmentation=augmentation,
            layers="4+")


Starting at epoch 70. LR=0.0003333333333333333

Checkpoint Path: /home/useydaninja/projects/melanoma/Skin-Cancer-Segmentation/logs/moles20200728T1523/mask_rcnn_moles_{epoch:04d}.h5
Selecting layers to train
res4a_branch2a         (Conv2D)
bn4a_branch2a          (BatchNorm)
res4a_branch2b         (Conv2D)
bn4a_branch2b          (BatchNorm)
res4a_branch2c         (Conv2D)
res4a_branch1          (Conv2D)
bn4a_branch2c          (BatchNorm)
bn4a_branch1           (BatchNorm)
res4b_branch2a         (Conv2D)
bn4b_branch2a          (BatchNorm)
res4b_branch2b         (Conv2D)
bn4b_branch2b          (BatchNorm)
res4b_branch2c         (Conv2D)
bn4b_branch2c          (BatchNorm)
res4c_branch2a         (Conv2D)
bn4c_branch2a          (BatchNorm)
res4c_branch2b         (Conv2D)
bn4c_branch2b          (BatchNorm)
res4c_branch2c         (Conv2D)
bn4c_branch2c          (BatchNorm)
res4d_branch2a         (Conv2D)
bn4d_branch2a          (BatchNorm)
res4d_branch2b         (Conv2D)
bn4d_branch2b          

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "







Epoch 71/90
400/400 [==============================] - 1035s 3s/step - loss: 0.7166 - val_loss: 0.6597

Epoch 72/90
400/400 [==============================] - 721s 2s/step - loss: 0.7285 - val_loss: 0.8465
Epoch 73/90
400/400 [==============================] - 919s 2s/step - loss: 0.7399 - val_loss: 0.9124
Epoch 74/90
400/400 [==============================] - 909s 2s/step - loss: 0.7095 - val_loss: 0.7878
Epoch 75/90
400/400 [==============================] - 912s 2s/step - loss: 0.7043 - val_loss: 0.6524
Epoch 76/90
400/400 [==============================] - 914s 2s/step - loss: 0.7093 - val_loss: 0.5876
Epoch 77/90
400/400 [==============================] - 916s 2s/step - loss: 0.6819 - val_loss: 0.7663
Epoch 78/90
400/400 [==============================] - 909s 2s/step - loss: 0.7013 - val_loss: 1.2000
Epoch 79/90
400/400 [==============================] - 903s 2s/step - loss: 0.7014 - val_loss: 0.5188
Epoch 80/90
400/400 [==============================] - 928s 2s/step - loss:

In [10]:
# Train all
model.train(ds, dataset_val,
            learning_rate=config.LEARNING_RATE/10,
            epochs=150,
            augmentation=augmentation,
            layers="all")


Starting at epoch 120. LR=0.0001

Checkpoint Path: /home/useydaninja/projects/melanoma/Skin-Cancer-Segmentation/logs/moles20200728T1523/mask_rcnn_moles_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
re

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "







Epoch 121/150
400/400 [==============================] - 872s 2s/step - loss: 0.4320 - val_loss: 1.2500

Epoch 122/150
400/400 [==============================] - 710s 2s/step - loss: 0.4228 - val_loss: 0.9005
Epoch 123/150
400/400 [==============================] - 722s 2s/step - loss: 0.4323 - val_loss: 0.4372
Epoch 124/150
400/400 [==============================] - 721s 2s/step - loss: 0.4077 - val_loss: 0.7600
Epoch 125/150
400/400 [==============================] - 722s 2s/step - loss: 0.4140 - val_loss: 0.5543
Epoch 126/150
400/400 [==============================] - 723s 2s/step - loss: 0.4091 - val_loss: 1.5256
Epoch 127/150
400/400 [==============================] - 721s 2s/step - loss: 0.3969 - val_loss: 1.1338
Epoch 128/150
400/400 [==============================] - 722s 2s/step - loss: 0.4115 - val_loss: 0.6331
Epoch 129/150
400/400 [==============================] - 721s 2s/step - loss: 0.4051 - val_loss: 0.3087
Epoch 130/150
400/400 [==============================] - 7

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Train top layers
model.train(ds, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=170, 
            layers='heads',
            augmentation=augmentation
            )


Starting at epoch 160. LR=0.001

Checkpoint Path: /home/useydaninja/projects/melanoma/Skin-Cancer-Segmentation/logs/moles20200728T1523/mask_rcnn_moles_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_c

/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/opt/conda/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "







Epoch 161/170
399/400 [============================>.] - ETA: 1s - loss: 0.8739